# Test notebook for augmenting a dataset from a local pickle that was saved from a pandas dataframe

In [1]:
import os
import sys
import numpy as np 
import open3d as o3d
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
import collections
import tensorflow_datasets as tfds
from tensorflow_graphics.geometry import transformation

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


2024-07-29 09:08:27.215317: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-29 09:08:27.588105: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-29 09:08:27.590683: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-29 09:08:28.927444: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
# load pandas dataframe from pickle
df = pd.read_pickle("../df_jrdb.pkl") 
df=df.reset_index(drop=True)

In [3]:
df

,positions,poses
0,"[[1.6433, 1.31074, 0.05498], [1.61661, 1.27148...","[[[21.4, 9.8, 2.0], [25.55, 31.25, 2.0], [38.1..."
1,"[[0.83958, -0.50863, 0.045], [0.85848, -0.4422...","[[[563.57, -178.91, 0.0], [587.46, -136.51, 0...."
2,"[[1.53163, 1.33974, 0.03798], [1.52811, 1.3346...","[[[11.24, 23.85, 2.0], [-2.52, 46.95, 0.0], [-..."
3,"[[1.84136, 1.52542, -0.005], [1.83469, 1.50237...","[[[60.4, 70.02, 2.0], [54.05, 88.92, 2.0], [58..."
4,"[[1.72412, 1.30398, -0.01211], [1.69748, 1.244...","[[[68.25, 39.05, 2.0], [69.55, 66.05, 2.0], [7..."
...,...,...
62,"[[0.58309, 1.53873, -0.06542], [0.56544, 1.534...","[[[355.9, 57.25, 1.0], [357.35, 81.8, 0.0], [3..."
63,"[[-1.21258, 3.05561, 0.0253], [-1.06531, 3.124...","[[[34.9, 98.15, 2.0], [34.2, 114.9, 0.0], [25...."
64,"[[3.84866, 2.06216, -0.06405], [3.82804, 2.031...","[[[721.26, 137.3, 2.0], [714.6, 147.4, 0.0], [..."
65,"[[1.87661, 1.81481, -0.10096], [1.87932, 1.813...","[[[592.85, 118.5, 1.0], [605.65, 136.4, 0.0], ..."


In [14]:
def random_rotate(df):
    """
    Randomly rotates a 3D coordinate in the 2D ground plane 
    """
    augmented_df = pd.DataFrame(columns=['positions', 'poses'])

    for i,row in df.iterrows():
        yaw = np.random.uniform(-np.pi, np.pi)
        
        # Create rotation matrix for the yaw angle
        cos_yaw = np.cos(yaw)
        sin_yaw = np.sin(yaw)
        rot_mat = np.array([
            [cos_yaw, -sin_yaw, 0],
            [sin_yaw,  cos_yaw, 0],
            [      0,        0, 1]
        ])

        augmented_df.loc[i] = [np.dot(row['positions'], rot_mat.T), np.dot(row['poses'], rot_mat.T)]
    return augmented_df

def random_translate(df):
    """
    Randomly translates a 3D coordinate in the 2D ground plane 
    """
    augmented_df = pd.DataFrame(columns=['positions', 'poses'])

    for i,row in df.iterrows():
        translation = np.random.uniform(-10.0, 10.0, 2)
        translation = np.append(translation, [0.0])
        
        augmented_df.loc[i] = [row['positions'] + translation, row['poses']]

    return augmented_df


new_df = random_rotate(df)
new_df = random_translate(new_df)

#new_df = pd.DataFrame.from_dict(new_dict)

In [15]:
new_df

,positions,poses
0,"[[-3.4380422709167577, 5.496500158942387, 0.05...","[[[11.657997306037991, 20.447276073169522, 2.0..."
1,"[[-0.8680583536134464, -9.927530091425838, 0.0...","[[[-345.96087821211677, -479.5112133690941, 0...."
2,"[[-10.294995405306327, -0.37889119509966696, 0...","[[[-16.61921320153698, -20.468557657095893, 2...."
3,"[[-4.8761198990390024, 5.964402314525507, -0.0...","[[[90.8391912559368, 17.297448712666707, 2.0],..."
4,"[[-1.881092878829345, 5.725830992093375, -0.01...","[[[54.859678025656, -56.332767790348775, 2.0],..."
...,...,...
2142,"[[0.4630899066480936, -3.0064596616022112, -0....","[[[170.1467975659161, -317.7930769825912, 1.0]..."
2143,"[[2.4524027723438007, -0.6039926814975514, 0.0...","[[[102.86054409716029, -16.466358675680063, 2...."
2144,"[[-13.182712798793832, -9.855352794527354, -0....","[[[-672.2220528842901, -295.27070497431805, 2...."
2145,"[[-8.080790470831579, 6.350803512983255, -0.10...","[[[450.3419367635172, 403.3677137452678, 1.0],..."


In [17]:
# the original dataframe with the augmented one
aug_df = pd.concat([df, new_df], ignore_index=True)
aug_df

,positions,poses
0,"[[1.6433, 1.31074, 0.05498], [1.61661, 1.27148...","[[[21.4, 9.8, 2.0], [25.55, 31.25, 2.0], [38.1..."
1,"[[0.83958, -0.50863, 0.045], [0.85848, -0.4422...","[[[563.57, -178.91, 0.0], [587.46, -136.51, 0...."
2,"[[1.53163, 1.33974, 0.03798], [1.52811, 1.3346...","[[[11.24, 23.85, 2.0], [-2.52, 46.95, 0.0], [-..."
3,"[[1.84136, 1.52542, -0.005], [1.83469, 1.50237...","[[[60.4, 70.02, 2.0], [54.05, 88.92, 2.0], [58..."
4,"[[1.72412, 1.30398, -0.01211], [1.69748, 1.244...","[[[68.25, 39.05, 2.0], [69.55, 66.05, 2.0], [7..."
...,...,...
4289,"[[0.4630899066480936, -3.0064596616022112, -0....","[[[170.1467975659161, -317.7930769825912, 1.0]..."
4290,"[[2.4524027723438007, -0.6039926814975514, 0.0...","[[[102.86054409716029, -16.466358675680063, 2...."
4291,"[[-13.182712798793832, -9.855352794527354, -0....","[[[-672.2220528842901, -295.27070497431805, 2...."
4292,"[[-8.080790470831579, 6.350803512983255, -0.10...","[[[450.3419367635172, 403.3677137452678, 1.0],..."


In [18]:
aug_df.to_pickle('df_jrdb_augmented.pkl')